In [25]:
from deltalake.writer import write_deltalake
from deltalake import DeltaTable
import duckdb
import pyarrow.parquet as pq

In [29]:
cn = duckdb.connect(
)
cn.sql("""from duckdb_extensions() """)

┌──────────────────┬─────────┬───────────┬─────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────┬───────────────────┬───────────────────┬───────────────────┬────────────────┐
│  extension_name  │ loaded  │ installed │                                  install_path                                   │                                    description                                     │      aliases      │ extension_version │   install_mode    │ installed_from │
│     varchar      │ boolean │  boolean  │                                     varchar                                     │                                      varchar                                       │     varchar[]     │      varchar      │      varchar      │    varchar     │
├──────────────────┼─────────┼───────────┼─────────────────────────────────────────────────────────────────────────────────┼───────────────

In [30]:
# Criando uma tabela e inserindo dados
cn.execute("""
CREATE TABLE clientes (
    id INTEGER,
    nome TEXT,
    idade INTEGER
);
""")

In [32]:
cn.execute("""
INSERT INTO clientes VALUES 
    (1, 'João', 30),
    (2, 'Maria', 25),
    (3, 'Carlos', 40);
""")


In [43]:
# Consultando os dados
clientes_df = cn.execute("SELECT * FROM clientes;").fetchdf()
print(clientes_df)
cn.table("clientes").show()

   id    nome  idade
0   1    João     30
1   2   Maria     25
2   3  Carlos     40
┌───────┬─────────┬───────┐
│  id   │  nome   │ idade │
│ int32 │ varchar │ int32 │
├───────┼─────────┼───────┤
│     1 │ João    │    30 │
│     2 │ Maria   │    25 │
│     3 │ Carlos  │    40 │
└───────┴─────────┴───────┘



In [41]:
cn.execute("show tables;").fetchdf()

cn.table("clientes").show()

┌───────┬─────────┬───────┐
│  id   │  nome   │ idade │
│ int32 │ varchar │ int32 │
├───────┼─────────┼───────┤
│     1 │ João    │    30 │
│     2 │ Maria   │    25 │
│     3 │ Carlos  │    40 │
└───────┴─────────┴───────┘



In [45]:
# Lendo o arquivo Parquet
cn.execute("COPY clientes TO 'clientes.parquet' (FORMAT 'parquet')");

In [46]:
df_parquet = pq.read_table('clientes.parquet').to_pandas()
print(df_parquet.head())

   id    nome  idade
0   1    João     30
1   2   Maria     25
2   3  Carlos     40


In [64]:
cn.execute("CREATE TABLE readings_stg AS  SELECT *  FROM read_json('https://environment.data.gov.uk/flood-monitoring/data/readings')");

cn.execute("CREATE TABLE measures_stg AS  SELECT *  FROM read_json('https://environment.data.gov.uk/flood-monitoring/id/measures')");

cn.execute("CREATE TABLE stations_stg AS  SELECT * FROM read_json('https://environment.data.gov.uk/flood-monitoring/id/stations')");

cn.execute("CREATE TABLE floods_stg AS  SELECT *  FROM read_json('https://environment.data.gov.uk/flood-monitoring/id/floods')");

cn.execute("CREATE TABLE floodAreas_stg AS  SELECT *  FROM read_json('https://environment.data.gov.uk/flood-monitoring/id/floodAreas')");


CatalogException: Catalog Error: Table with name "readings_stg" already exists!

In [65]:
cn.execute("show tables;").fetchdf()

,name
0,clientes
1,floodAreas_stg
2,floods_stg
3,measures_stg
4,readings_stg
5,stations_stg


In [56]:
cn.execute("describe readings_stg;").fetchdf()

,column_name,column_type,null,key,default,extra
0,@context,VARCHAR,YES,None,None,None
1,meta,"STRUCT(publisher VARCHAR, licence VARCHAR, doc...",YES,None,None,None
2,items,"STRUCT(""@id"" VARCHAR, dateTime TIMESTAMP, meas...",YES,None,None,None


In [89]:
cn.execute("SELECT * FROM  readings_stg r ;").fetchall()

[('http://environment.data.gov.uk/flood-monitoring/meta/context.jsonld',
  {'publisher': 'Environment Agency',
   'licence': 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/',
   'documentation': 'http://environment.data.gov.uk/flood-monitoring/doc/reference',
   'version': '0.9',
   'comment': 'Status: Beta service',
   'hasFormat': ['http://environment.data.gov.uk/flood-monitoring/data/readings.csv',
    'http://environment.data.gov.uk/flood-monitoring/data/readings.rdf',
    'http://environment.data.gov.uk/flood-monitoring/data/readings.ttl',
    'http://environment.data.gov.uk/flood-monitoring/data/readings.html'],
   'limit': 500},
  [{'@id': 'http://environment.data.gov.uk/flood-monitoring/data/readings/1029TH-level-downstage-i-15_min-mASD/2025-02-15T00-00-00Z',
    'dateTime': datetime.datetime(2025, 2, 15, 0, 0),
    'measure': 'http://environment.data.gov.uk/flood-monitoring/id/measures/1029TH-level-downstage-i-15_min-mASD',
    'value': -0.14},
   {'

In [90]:
cn.execute("SELECT * FROM measures_stg m; ").fetchall()

[('http://environment.data.gov.uk/flood-monitoring/meta/context.jsonld',
  {'publisher': 'Environment Agency',
   'licence': 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/',
   'documentation': 'http://environment.data.gov.uk/flood-monitoring/doc/reference',
   'version': '0.9',
   'comment': 'Status: Beta service',
   'hasFormat': ['http://environment.data.gov.uk/flood-monitoring/id/measures.csv',
    'http://environment.data.gov.uk/flood-monitoring/id/measures.rdf',
    'http://environment.data.gov.uk/flood-monitoring/id/measures.ttl',
    'http://environment.data.gov.uk/flood-monitoring/id/measures.html']},
  [{'@id': 'http://environment.data.gov.uk/flood-monitoring/id/measures/1029TH-level-downstage-i-15_min-mASD',
    'datumType': 'http://environment.data.gov.uk/flood-monitoring/def/core/datumASD',
    'label': 'RIVER DIKLER AT BOURTON ON THE WATER - level-downstage-i-15_min-mASD',
    'latestReading': {'@id': 'http://environment.data.gov.uk/flood-monit